In [19]:
from datetime import datetime
from datetime import timedelta
from random import randint, choice

import pandas as pd
from sqlalchemy import create_engine

from settings.settings import CUSTOMERS_TABLE_FIELDS, ITEMS_TABLE_FIELDS, TRANSACTIONS_TABLE_FIELDS

engine = create_engine("postgres://miningAgent:@localhost/QeNoBi")

In [20]:
### Generate transactions
nb_transactions = 10000
customers_range = (0,50)
items_range= (0,100)
stations_range = (0,100)
date_min = datetime(2017,1,1)
max_days = 356
res = []
for i in range(0,nb_transactions):
    res.append((
        randint(*items_range),
        randint(*customers_range),
        randint(*stations_range),
        (date_min + timedelta(days=randint(10,max_days))),
        i,
    ))
transactions = pd.DataFrame(res,columns=TRANSACTIONS_TABLE_FIELDS)
transactions["transaction_date"] = pd.to_datetime(transactions["transaction_date"])
transactions.to_sql("transactions",con=engine,if_exists="replace",index=False,method="multi",chunksize=2000)

In [21]:
transactions

,item_id,customer_id,station_id,transaction_date,transaction_id
0,28,39,74,2017-07-08,0
1,10,10,68,2017-05-10,1
2,9,31,78,2017-09-09,2
3,95,25,99,2017-10-24,3
4,97,18,81,2017-12-12,4
...,...,...,...,...,...
9995,22,0,16,2017-12-02,9995
9996,81,24,72,2017-01-13,9996
9997,44,18,91,2017-09-01,9997
9998,92,10,80,2017-04-15,9998


In [22]:
###### Insertion of customers
sex_values = ["M","F"]
age_values = ["<35","35-49",">49"]
departements_values = [i for i in range(0,98)]
res = []
for i in transactions.customer_id.unique():
    res.append((
        i,
        choice(sex_values),
        choice(age_values),
        choice(departements_values)
    ))
customers = pd.DataFrame(res,columns=CUSTOMERS_TABLE_FIELDS)
customers.to_sql("customers",con=engine,if_exists="replace",index=False)

In [23]:
### Insertion of items
items_columns = ["description","item_id"]
res = []
for i in transactions.item_id.unique():
    res.append((
        f"I-{i}",
        i,
    ))
items_df = pd.DataFrame(res,columns=ITEMS_TABLE_FIELDS)
items_df.to_sql('items',con=engine,if_exists="replace",index=False)

In [24]:
#### Insertion of stations
stations_columns = ["station_id","station_type"]
res = []
for i in transactions.station_id.unique():
    res.append((
        i,
        f"TYPE-{i%3}"
    ))
stations = pd.DataFrame(res,columns=stations_columns)
stations.to_sql("stations",con=engine,if_exists="replace",index=False)
